실습을 시작하기 전에, 코드가 올바른 폴더에 위치해 있는지 점검해보자.

**BERT 파일 전체는 반드시 개인 구글 드라이브 내 BERT 폴더에 위치해 있어야 한다.**

뒤에 ratings.txt를 읽어오는데 어려움을 겪고 있다면, 파일들이 올바른 위치에 있는지 다시 한 번 점검해보자.

# **Preproc.ipynb**

우리는 문장을 입력해주었을 때, 그 문장이 긍정문인지 부정문인지 판단해주는 AI 모델을 구축하고자 한다. AI 모델이 문장을 올바르게 분류하기 위해서는 문장 내 정보들을 인지하는 과정이 필요하다.

컴퓨터는 우리가 사용하는 언어를 있는 그대로 받아들일 수 없다. 컴퓨터가 이해할 수 있는 방식, 숫자로 변환해주어야 한다. 이와 같이 AI 모델이 주어진 데이터를 이해할 수 있도록, 데이터의 형태를 변환해주는 과정을 전처리 (Preprocessing) 라고 한다.

**본 파일에서는 문장 데이터를 전처리하는 과정에 대해 다뤄볼 것이다.**

전처리에 필요한 함수들을 구성한 뒤, 최하단의 preproc_test 함수를 통해서 테스트해볼 것이다.

연습 문제를 시작하기에 앞서, 필요한 라이브러리들을 설치하자.

연습 문제에 필요한 라이브러리들을 import하자.

In [29]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
# from keras.utils import pad_sequences
import pandas as pd
import traceback

Pandas 라이브러리의 read_csv 함수를 활용하여 데이터셋을 읽어오자.

In [30]:
whole_dataset = pd.read_csv('./ratings.txt', delimiter="\t")

In [31]:
whole_dataset.columns

Index(['id', 'document', 'label'], dtype='object')

읽어 온 데이터의 형태는 어떠할까?

head 함수를 활용하여 상단 5개의 data들을 출력해보고, data의 row와 column은 어떻게 구성되어 있는지 살펴보자.

In [32]:
whole_dataset.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [33]:
whole_dataset['label']

0         1
1         1
2         1
3         1
4         1
         ..
199995    0
199996    0
199997    0
199998    0
199999    0
Name: label, Length: 200000, dtype: int64

데이터를 살펴보니, row는 0부터 차례로 이어지며, column은 id, document, label로 구성되어 있다.

id는 데이터를 구분짓기 위한 일련 번호, document는 문장 데이터, label은 해당하는 문장이 긍정(1), 부정(0)인지 나타내는 라벨 값이다.

여기서 우리는 학습 데이터들 간에 구분할 필요가 없으므로, id 데이터는 분리하자. 그리고 column 이름으로 document보다는 sentence가 적절해보이니, 그렇게 바꿔보자.

In [73]:
def data_processing(raw_data):
    transformed_data = raw_data['label']

    # 판다스의 concat을 활용하여 'document' 데이터와 'label' 데이터를 연결해보자.
    # [1] new_array = pandas.concat([array1, array2])를 하면, array1과 array2가 물리적으로 연결된 new_array를 생성할 수 있다.
    #     이때 추가 옵션을 별도로 지정하지 않았으므로, concat을 수행하는 default 방향은 axis=0(default)이다.
    # [2] concat 시 axis 조건을 통해 array를 concat하는 방향을 직접 지정해주도록 하자.
    #     Hint: 2차원 array에서 axis=0은 세로(상/하) 방향, axis=1은 가로(좌/우) 방향이다.
    #     우리는 'document'열과 'label' 열을 가로(좌/우) 방향으로 concat해야 하므로 concat 시 axis=1이라는 조건을 설정하는 것이 적합할 것이다.
    # concat에 대한 추가적인 내용은 https://yganalyst.github.io/data_handling/Pd_12/ 를 참고해보자.
    # axis에 대한 추가적인 내용은 https://jalammar.github.io/visual-numpy/ https://lets-hci-la-ai-withme.tistory.com/15 를 참고해보자.
    ## 여기에 코드 작성
    raw_data['document'].astype('str')
    processed_data = pd.concat([raw_data['document'], transformed_data], axis=1)

    # 데이터의 column 이름을 sentence label로 바꾸어준다.
    processed_data.columns = ['sentence', 'label']

    return processed_data

여기까지 구현한 부분을 하단의 preproc_test 함수의 첫 번째 테스트로 확인해볼 것이다. (20점)

이제 문장 데이터를 본격적으로 변환시켜볼 것이다.

아래 data_to_token_ids 함수를 아래와 같은 절차를 통해 문장 데이터를 수치적으로 변환한다.

1. Tokenizer가 문장의 시작과 끝을 인식할 수 있도록 문장 앞뒤에 CLS 토큰과 SEP 토큰을 붙인다.

2. Tokenizer의 tokenize 함수를 활용하여 문장을 여러 개의 토큰으로 나눈다.

3. Tokenizer의 convert_tokens_to_ids 함수를 활용하여, 토큰들을 대응되는 id로 변환해준다.

4. MAX_LEN의 길이에 맞춰, padding을 진행해준다. 비어있는 자리의 경우, 0이 입력된다.

In [61]:
def data_to_token_ids(tokenizer, single_sentence):
    # CLS 토큰과 SEP 토큰을 문장의 시작과 끝에 붙여보자.
    ## 여기에 코드 작성
    special_token_added =  '[CLS]' + single_sentence + '[SEP]' # 예상 
    # tokenizer의 tokenize 함수를 활용하여 문장을 토큰화해보자.
    tokenized_sentence = tokenizer.tokenize(special_token_added) # 에러 

    # tokenizer의 convert_tokens_to_ids 함수를 활용하여 생성된 토큰을 숫자 형태로 바꿔주자.
    token_ids = [tokenizer.convert_tokens_to_ids(tokenized_sentence)]

    MAX_LEN = 128
    # pad_sequences 함수를 활용하여 문장의 빈 칸에 padding을 넣어주자.
    # keras의 preprocessing.sequence 라이브러리는 pad_sequences 함수를 제공하며, 본 함수는 서로 다른 길이의 문장을 특정 길이(최대 길이)로 일치시키기 위해 truncating와 padding을 한다.
    # truncating은 최대 길이보다 긴 문장을 최대 길이에 맞게 잘라내는 것이고, padding은 최대 길이보다 짧은 문장의 남는 자리를 0으로 채우는 것을 가리킨다.
    # pad_sequence는 default 옵션은 'pre'이다. 즉 긴 문장의 앞 부분을 잘라내거나 짧은 문장의 앞에 0을 채운다.
    # 그러나 우리는 토큰 id 리스트의 '뒷'부분에 truncating 및 padding을 적용해주고자 한다.
    # truncating="post" 옵션을 통해 id 리스트의 길이가 MAX_LEN을 넘어가는 부분에 대해서는 뒷부분을 삭제할 수 있다.
    # padding="post" 옵션을 통해 토큰 id 리스트의 뒷부분에 padding을 적용할 수 있다.
    # 최대 길이는 위의 MAX_LEN으로 설정하고, dtype은 long으로 설정해주자.
    # 위 설정에 맞게 padding, truncating 값을 적절히 설정해주자.
    # https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences
    ## 여기에 코드 작성
    token_ids_padded = pad_sequences(token_ids, maxlen=MAX_LEN, truncating='post', padding='post', dtype='long')


    token_ids_flatten = token_ids_padded.flatten()
    return token_ids_flatten

https://github.com/SKTBrain/KoBERT/issues/11

완성한 data_to_token_ids 함수를 간단히 활용해보며 함수의 반환값 형태를 익혀보자.  
문장이 토큰화되고 각 토큰이 id값으로 반환되었으며, 최대 길이에 미치지 못하는 부분은 문장의 뒷부분에 0으로 padding 처리가 되었음을 확인할 수 있다.  



```
from tokenization import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

id_testing = data_to_token_ids(tokenizer, "찐배고픔이랑 가짜배고픔이랑 구분하는건 이미포기했어")

print(id_testing)

# [   2  517 7385 6312 5439 7766 7096 6022  770 6312 5439 7766 7096 6022
 1115 6416 7794 5384 3692 7728 5561 7864 6855    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]

```

위와 같은 방식으로 tokenizer와 임의의 한국어 문장을 입력하여, 함수가 정확히 만들어졌는지 확인해보도록 하자.


In [49]:
from tokenization import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

id_testing = data_to_token_ids(tokenizer, "찐배고픔이랑 가짜배고픔이랑 구분하는건 이미포기했어")

print(id_testing)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


[   2  517 7385 6312 5439 7766 7096 6022  770 6312 5439 7766 7096 6022
 1115 6416 7794 5384 3692 7728 5561 7864 6855    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


pad_sequences 함수를 통해 padding이 이루어진 부분은 학습하는데 실질적으로 쓰이지 않는다.

그러므로, padding된 부분은 고려하지 않도록 필터링해주는 mask를 만들어보자.

여기서 mask는 padding된 부분은 지우고, 나머지 부분은 그대로 두는 기능을 한다. 고로, padding된 부분은 0, 아닌 부분은 1의 값을 나타내도록 한다.

In [37]:
def token_ids_to_mask(token_ids):

    # 한 문장에 대한 token_id 리스트를 입력으로 받는다.
    # token_id에서 0보다 큰 숫자만 유효하도록 하는 'mask' 리스트를 만들자.
    # 이 때, mask의 각 원소는 0 아니면 1의 값을 가져야 한다.
    # Hint : 각 token_id를 0 아니면 1의 값으로 바꿔주면 된다.
    # Hint : list comprehension을 활용해서 작성하면 편하다.
    # HINT : list comprehension을 활용하는 것이 가장 간결한 코드이며, 조금 어렵다면 for 문을 활용해봐도 좋다.
    ## 여기에 코드 작성
    mask = [0 if i <= 0 else 1 for i in token_ids]

    return mask

마찬가지로, 완성한 token_ids_to_mask 함수를 간단히 활용해보며 함수의 반환값 형태를 익혀보자.  
0으로 패딩처리된 부분은 0.0으로, 나머지는 1.0으로 채워진 max_length 길이의 array가 반환됨을 확인할 수 있다.

```
mask_testing = token_ids_to_mask(id_testing)
print(mask_testing)

# [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
```

위와 같은 방식으로, 앞서 생성한 id_testing을 입력하여 함수가 정확히 만들어졌는지 확인해보도록 하자.


In [38]:
mask_testing = token_ids_to_mask(id_testing)
print(mask_testing)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


data_to_token_ids 함수는 하나의 문장이 입력되었을 때, token_id 리스트를 출력해주는 함수이며, token_ids_to_mask 함수는 한 문장에 대한 token_id 리스트가 입력되었을 때, 하나의 mask를 생성해주는 함수다.
위에서 구성한 두 가지 함수를 활용해서, 전체 데이터를 변형하는 로직을 구성해보자.

*HINT : list comprehension을 활용하는 것이 가장 간결한 코드이며, 조금 어렵다면 for 문을 활용해봐도 좋다.*

In [67]:
# tokenize_processed_data 함수는, 앞서 정의한 함수인 data_to_token_ids와 token_ids_to_mask를 모두 활용한다.
# tokenize_processed_data 함수를 통해, 데이터를 구성하는 각 문장을 '토큰 id로 구성된, max_length 길이의 array'로 변환해주고, attention 마스크를 생성하는 작업을 한꺼번에 처리할 수 있다.
# 함수의 입력은 두 가지로, [1]tokenizer, [2] raw_data를 data_processing 함수로 전처리한 결과인 processed_dataset이다.
# 함수의 출력은 세 가지로, [1]tokenized_data (='토큰 id로 구성된 max_length 길이의 array'로 구성된 list), [2] 데이터의 labels, [3] 각 array에 대응하는 attention_masks로 구성된 list이다.

def tokenize_processed_data(tokenizer, processed_dataset):
    labels = processed_dataset['label'].to_numpy()

    # list comprehension을 활용하여 processed_dataset의 'sentence' 데이터를 id 리스트로 토큰화하자.
    ## 여기에 코드 작성
    tokenized_data = [data_to_token_ids(tokenizer, str(sentence)) for sentence in processed_dataset['sentence']]

    # list comprehension을 활용하여 앞서 토큰화한 id 리스트 각각을 mask로 변환하자.
    ## 여기에 코드 작성
    attention_masks = [token_ids_to_mask(data) for data in tokenized_data]

    return tokenized_data, labels, attention_masks

여기까지 구현한 부분을 하단의 preproc_test 함수의 두 번째 테스트로 확인해볼 것이다. (40점)

위의 함수를 통해 변환한 데이터셋을 train, validation, test용으로 나눠야 한다.

split_into_train_test 함수를 통해 train 데이터와 test 데이터로 나누고, 그렇게 나누어진 train 데이터를 split_into_train_validation 함수를 통해 train 데이터와 validation 데이터로 나눌 것이다.

sklearn.model_selection 라이브러리의 train_test_split 함수를 활용하여 아래의 두 함수를 구현해보자.

In [90]:
def split_into_train_test(whole_data, whole_label, whole_masks):
    print("length of whole_data : " + str(len(whole_data)))

    train_inputs, test_inputs, train_labels, test_labels = train_test_split(whole_data,
                                                                                    whole_label,
                                                                                    random_state=2022,
                                                                                    test_size=0.1)
    # 위의 방식을 참조하여 mask 역시 train을 위한 mask와 test을 위한 mask로 나누자.
    # 이때 return 값을 참조하여, 우리에게 불필요한 정보는 _로 비워두자.
    # random_state와 test_size 동일하게 설정
    ## 여기에 코드 작성
    train_masks, test_masks, _, _ = train_test_split(whole_masks, whole_data, random_state=2022, test_size=0.1)

    return train_inputs, test_inputs, train_labels, test_labels, train_masks, test_masks

In [91]:
def split_into_train_validation(train_data, train_label, train_masks):
    print("length of train_data : " + str(len(train_data)))

    # split_into_train_test의 코드를 참조하여 data와 mask 각각을 train을 위한 것과 validation을 위한 것으로 나누자.
    # random_state = 2022, test_size = 0.1로 설정

    ## 여기에 코드 작성(data)
    train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_data, train_label,
                                                                          random_state=2022,
                                                                          test_size=0.1)

    ## 여기에 코드 작성(mask)
    train_masks, validation_masks, _, _ = train_test_split(train_masks, train_data,
                                                            random_state=2022,
                                                            test_size=0.1)

    return train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks

여기까지 구현한 부분을 하단의 preproc_test 함수의 세 번째 테스트로 확인해볼
것이다. (60점)

우리는 이렇게 나누어진 데이터들을 tensor의 형태로 변환해주어야 한다. data_to_tensor는 그러한 역할을 해주는 함수다.

torch.tensor 함수를 활용해서 inputs, labels, masks 각각을 tensor로 변환해주는 함수를 구현해보자.

In [42]:
def data_to_tensor(inputs, labels, masks):
    # 입력받은 데이터를 텐서로 변환해주는 함수
    ## 여기에 코드 작성
    inputs_tensor = torch.tensor(inputs)
    labels_tensor = torch.tensor(labels)
    masks_tensor = torch.tensor(masks)
    return inputs_tensor, labels_tensor, masks_tensor

여기까지 구현한 부분을 하단의 preproc_test 함수의 네 번째 테스트로 확인해볼 것이다. (80점)

tensor로 변환한 데이터를 dataloader를 활용해서 batch 단위로 묶어줄 것이다.

batch로 데이터를 묶어주기 전에, 데이터를 어떤 순서로 뽑을 것인지 적절한 Sampler를 설정해주어야 한다. 데이터가 학습을 위한 것인지, 검증을 위한 것인지에 따라서 알맞은 Sampler를 배정해주도록 하자.

In [43]:
# 모든 DataLoader는 Sampler를 갖고 있다. Sampler는 데이터를 load 해올 때 데이터의 index를 컨트롤함으로써 어떤 데이터부터 가져올지 지정하는 기능을 한다.
# - SequentialSampler: 항상 같은 순서로, 순차적으로 데이터를 load 한다.
# - RandomSampler: 랜덤하게 데이터를 load 한다.

def tensor_to_dataloader(inputs, labels, masks, mode):
    from torch.utils.data import RandomSampler, SequentialSampler

    batch_size=32
    data = TensorDataset(inputs, masks, labels)

    if mode == "train":
        # train 모드에서는 랜덤하게 데이터를 load해오는 sampler를 사용하자.
        # 대개 mini-batch 내부 구성이 다양할수록 전체 dataset(모집단)를 잘 대표하기 때문에 주로 RandomSampler를 사용한다. (https://hul980.tistory.com/28)
        ## 여기에 코드 작성
        datasampler = RandomSampler(data)
    else:
        # test에는 순차적으로 데이터를 load하는 sampler을 지정하자.
        ## 여기에 코드 작성
        datasampler = SequentialSampler(data)

    # DataLoader 함수를 활용해서 dataloader를 선언해보자.
    # batch_size는 batch_size로 설정하고, sampler는 위에서 지정한대로 설정해주자.
    ## 여기에 코드 작성
    dataloader = DataLoader(data, batch_size=batch_size, sampler=datasampler)

    return dataloader

여기까지 구현한 부분을 하단의 preproc_test 함수의 다섯 번째 테스트로 확인해볼 것이다. (100점)

이제 전처리를 수행하는데 필요한 모든 함수들을 다 구현했다.

구현한 함수들을 모아 preproc 함수를 최종적으로 만들었다.

In [103]:
def preproc(tokenizer, whole_dataset):
    # whole_dataset을 전처리하자.
    processed_dataset = data_processing(whole_dataset)

    # 전처리한 전체 데이터를 토큰화하자.
    tokenized_dataset, labels, attention_masks = tokenize_processed_data(tokenizer, processed_dataset)

    # 토큰화한 전체 데이터를 train용과 test용으로 분리하자.
    train_inputs, test_inputs, train_labels, test_labels, train_masks, test_masks = split_into_train_test(tokenized_dataset, labels, attention_masks)
    # 토큰화한 train용 데이터를 train용과 validation용으로 분리하자.
    train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = split_into_train_validation(train_inputs, train_labels, train_masks)

    # train용, validation용, test용 데이터 각각을 텐서로 변환하자.
    train_inputs, train_labels, train_masks = data_to_tensor(train_inputs, train_labels, train_masks)
    validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_inputs, validation_labels, validation_masks)
    test_inputs, test_labels, test_masks = data_to_tensor(test_inputs, test_labels, test_masks)

    # train용, validation용, test용 텐서를 dataloader로 변환하자.
    train_dataloader = tensor_to_dataloader(train_inputs, train_labels, train_masks, "train")
    validation_dataloader = tensor_to_dataloader(validation_inputs, validation_labels, validation_masks, "validation")
    test_dataloader = tensor_to_dataloader(test_inputs, test_labels, test_masks, "test")

    return train_dataloader, validation_dataloader, test_dataloader

함수들이 잘 적절히 잘 만들어졌는지 preproc_test 함수를 통해 측정해보자.

5개의 테스트로 구성되어 있으며, 각 테스트는 20점이다.

첫 번째 테스트는 별도의 함수는 존재하지 않는다.

In [101]:
def test2(tokenized_data):
    real_data = [2, 1706, 6664, 5729, 6983,  517, 7990, 6493, 7828, 5943, 4928, 1861, 5783, 2235,
                 6527,   54, 7227, 6160, 3010, 6559, 7828, 2846, 7095, 3394, 6946,   54, 5782, 6150,
                 3093, 6653, 7010, 5384, 3647, 2846, 6116, 4147, 6441,  517, 5693, 5693, 7828, 4768,
                 5330,  743, 5451, 6903, 4147, 7869, 6198, 4102, 2034, 7170, 7792, 4709, 7879, 7328,
                 54, 1185, 6049, 5782, 5439, 5007, 3647, 2680, 5330, 3135, 7271, 5782, 5760, 5384,
                 1861, 3100,   54, 1569, 4196, 3093, 6653, 7013, 2571,   54,    3,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0]
    return (tokenized_data[2022] == real_data).all()


def test3(train_inputs, validation_inputs, test_masks):
    if len(train_inputs) != 162000 or len(validation_inputs) != 18000 or len(test_masks) != 20000:
        return False
    real_data = [2, 3765, 6954, 4207, 7850, 4446, 6395, 5761, 4102, 3977, 6881, 6701,   54, 2368,
                 517, 7265, 6827, 6701,   54,    3,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0]
    return (train_inputs[2022] == real_data).all()


def test4(train_inputs):
    real_input = torch.tensor([2, 3301, 6553, 6410,  517, 6193, 7591, 4179, 6141, 6255, 4244, 5439,
                               4012,  517, 6193, 7591, 1370, 5347, 5782, 5330, 2573, 6844, 7495, 1844,
                               6190, 1734, 6978, 7968, 7720, 7086,  517, 6193, 7591, 4179, 7788,  517,
                               6394, 5833, 6141, 7318, 6149, 7086, 3524, 7227, 5859, 7136, 5546, 5850,
                               2034, 7170, 7095, 1369, 5760, 1420,   55,    3,    0,    0,    0,    0,
                               0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                               0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                               0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                               0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                               0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                               0,    0,    0,    0,    0,    0,    0,    0])

    print(train_inputs[12345].dtype, real_input.dtype)
    return torch.equal(train_inputs[12345], real_input)


def test5(train_dataloader):
    real_input_ids = torch.tensor([2,  529,   54, 2860, 6295, 7640, 5371, 3594, 7837,  553,   54,  773,
                                   6383, 7095, 5037, 6645, 7837, 4501, 5957, 6629, 7288, 3714, 7207, 5357,
                                   589,   54, 2417, 5398, 6882, 3357,  631,  529, 7220,    3,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0])

    for step, batch in enumerate(tqdm(train_dataloader)):

        if step < 1234:
            continue
        if step > 1234:
            break

        b_input_ids, b_input_mask, b_labels = batch

    return torch.equal(b_input_ids[5], real_input_ids)

def preproc_test(tokenizer, whole_dataset):

    print("================={}번째 테스트 시작===================".format(1))
    # whole_dataset을 전처리하자.
    try:
      processed_dataset = data_processing(whole_dataset)
    except:
      print(traceback.format_exc())
      return 0
    print("================={}번째 테스트 성공===================\n".format(1))
    
    print(type(processed_dataset['sentence']), processed_dataset['sentence'].dtype)


    print("================={}번째 테스트 시작===================".format(2))
    # 전처리한 전체 데이터를 토큰화하자.
    try:
      tokenized_dataset, labels, attention_masks = tokenize_processed_data(tokenizer, processed_dataset)
    except:
      print(traceback.format_exc())
      return 20
    if not test2(tokenized_dataset):
      return 20
    print("================={}번째 테스트 성공===================\n".format(2))


    print("================={}번째 테스트 시작===================".format(3))
    # 토큰화한 전체 데이터를 train용과 test용으로 분리하자.
    try:
      train_inputs, test_inputs, train_labels, test_labels, train_masks, test_masks = split_into_train_test(tokenized_dataset, labels, attention_masks)
      # 토큰화한 train용 데이터를 train용과 validation용으로 분리하자.
      train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = split_into_train_validation(train_inputs, train_labels, train_masks)
    except:
      print(traceback.format_exc())
      return 40
    if not test3(train_inputs, validation_inputs, test_masks):
      return 40
    print("================={}번째 테스트 성공===================\n".format(3))


    print("================={}번째 테스트 시작===================".format(4))
    # train용, validation용, test용 데이터 각각을 텐서로 변환하자.
    try:
      train_inputs, train_labels, train_masks = data_to_tensor(train_inputs, train_labels, train_masks)
      validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_inputs, validation_labels, validation_masks)
      test_inputs, test_labels, test_masks = data_to_tensor(test_inputs, test_labels, test_masks)
    except:
      print(traceback.format_exc())
      return 60
    if not test4(train_inputs):
      return 60
    print("================={}번째 테스트 성공===================\n".format(4))


    print("================={}번째 테스트 시작===================".format(5))
    # train용, validation용, test용 텐서를 dataloader로 변환하자.
    try:
      train_dataloader = tensor_to_dataloader(train_inputs, train_labels, train_masks, "train")
      validation_dataloader = tensor_to_dataloader(validation_inputs, validation_labels, validation_masks, "validation")
      test_dataloader = tensor_to_dataloader(test_inputs, test_labels, test_masks, "test")
    except:
      print(traceback.format_exc())
      return 80
    if not test5(train_dataloader):
      return 80
    print("================={}번째 테스트 성공===================\n".format(5))


    return 100

In [52]:
def main():
    from tokenization import KoBertTokenizer
    # 전체 데이터를 불러오자.
    whole_dataset = pd.read_csv('ratings.txt', delimiter="\t")

    # KoBERTTokenizer를 불러오자.
    tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

    score = preproc_test(tokenizer, whole_dataset)
    print("현재 점수 : {}/100점".format(score))


아래 쉘을 실행하면 테스트의 점수를 알 수 있다.

***100점이 되어야 다음 섹터로 넘어갈 수 있다.***

In [102]:
if __name__ == '__main__':

    # 시드 고정
    seed_val = 2022
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    main()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


=================1번째 테스트 시작===================
=================1번째 테스트 성공===================

<class 'pandas.core.series.Series'> object
=================2번째 테스트 시작===================
=================2번째 테스트 성공===================

=================3번째 테스트 시작===================
length of whole_data : 200000
length of train_data : 180000
=================3번째 테스트 성공===================

=================4번째 테스트 시작===================
torch.int32 torch.int64


RuntimeError: Expected object of scalar type int but got scalar type __int64 for argument 'other'